In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
# --- CONFIGURATION ---
FILE_PATH = 'new_tmdb_movies_master.jsonl'  # Use your actual filename
START_DATE = '2015-01-01'
END_DATE = '2024-12-31'

# Streaming Keywords
STREAMING_KEYWORDS = [
    'Netflix', 'Amazon Studios', 'Hulu', 'Apple TV+', 'Disney+', 'HBO Max',
    'Prime Video', 'Paramount+', 'Warner Bros. Television', 'Sky'
]



In [3]:
data_list = []
with open(FILE_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line: continue
        try:
            data_list.append(json.loads(line))
        except json.JSONDecodeError:
            continue

# Create DataFrame
df_raw = pd.DataFrame(data_list)
print(f"Loaded {len(df_raw)} raw rows.")

Loaded 26191 raw rows.


In [4]:
df_raw['release_date'] = pd.to_datetime(df_raw['release_date'], errors='coerce')

# Filter for your Time Window
df_raw = df_raw[
    (df_raw['release_date'] >= START_DATE) & 
    (df_raw['release_date'] <= END_DATE)
].copy()

In [ ]:
df_raw['is_jan_1'] = (df_raw['release_date'].dt.month == 1) & (df_raw['release_date'].dt.day == 1)

print(f"\nArtifacts (Jan 1st releases): {df_raw['is_jan_1'].sum()}")



Potential 'Dump Date' Artifacts (Jan 1st releases): 2193
If this number is huge (e.g., >500), these are likely database defaults, not real releases.


In [6]:
cols_to_numeric = ['budget', 'revenue', 'vote_count', 'vote_average', 'popularity']
for col in cols_to_numeric:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce').fillna(0)

In [7]:
valid_budget = df_raw['budget'] > 1000  # Threshold to avoid micro-budget noise
df_raw['roi'] = np.where(valid_budget, (df_raw['revenue'] - df_raw['budget']) / df_raw['budget'], np.nan)

In [8]:
def get_main_company(companies):
    if isinstance(companies, list) and len(companies) > 0:
        return companies[0].get('name', 'Unknown')
    return 'Unknown'

In [9]:
df_raw['main_company'] = df_raw['production_companies'].apply(get_main_company)

In [10]:
def is_streaming_prod(companies):
    if not isinstance(companies, list): return False
    txt = " ".join([c.get('name', '').lower() for c in companies])
    return any(k.lower() in txt for k in STREAMING_KEYWORDS)

df_raw['is_streaming'] = df_raw['production_companies'].apply(is_streaming_prod)

In [11]:
print("\n--- Top 5 High Revenue but Low Rating (< 6.0) ---")
high_rev_low_rate = df_raw[
    (df_raw['revenue'] > 100_000_000) & 
    (df_raw['vote_average'] < 6.0)
].sort_values('revenue', ascending=False).head(5)
print(high_rev_low_rate[['title', 'release_date', 'revenue', 'vote_average', 'main_company']])

print("\n--- Top 5 Streaming Movies by Popularity ---")
streaming_hits = df_raw[df_raw['is_streaming']].sort_values('popularity', ascending=False).head(5)
print(streaming_hits[['title', 'release_date', 'vote_average', 'main_company']])

print("\n--- Dataframe ready as 'df_raw' ---")


--- Top 5 High Revenue but Low Rating (< 6.0) ---
                                    title release_date    revenue  \
80     Batman v Superman: Dawn of Justice   2016-03-23  874362803   
352                         Suicide Squad   2016-08-03  749200054   
3653                Detective Chinatown 3   2021-02-12  686257563   
13684                      Full River Red   2023-01-22  673556758   
85                   Fifty Shades of Grey   2015-02-11  569651467   

       vote_average                 main_company  
80            5.987        Warner Bros. Pictures  
352           5.919        Warner Bros. Pictures  
3653          5.600  Beijing Yitong Legend Films  
13684         5.920           Huanxi Media Group  
85            5.879           Universal Pictures  

--- Top 5 Streaming Movies by Popularity ---
                                              title release_date  \
971                               Top Gun: Maverick   2022-05-21   
3401  Mission: Impossible - Dead Reckoning Par

In [12]:
# 1. Filter the DataFrame for only the Jan 1st releases
jan_1_movies = df_raw[df_raw['is_jan_1']]

# 2. Define the metric for "success" (e.g., 'revenue', 'popularity', or 'vote_count')
success_metric = 'revenue'  # Change to 'popularity' if revenue data is sparse

# 3. Sort by the metric in descending order and take the top 20
if success_metric in jan_1_movies.columns:
    top_20_jan_1 = jan_1_movies.sort_values(by=success_metric, ascending=False).head(20)
    
    # 4. Select relevant columns for manual evaluation
    # Using a list comprehension to only select columns that actually exist in your dataframe
    columns_to_show = ['title', 'release_date', 'revenue', 'popularity', 'vote_count']
    existing_cols = [col for col in columns_to_show if col in top_20_jan_1.columns]
    
    print(f"Top 20 Jan 1st releases by {success_metric}:")
    print(top_20_jan_1[existing_cols])
else:
    print(f"Column '{success_metric}' not found in DataFrame.")


Top 20 Jan 1st releases by revenue:
                                  title release_date    revenue  popularity  \
2932                      The Gentlemen   2020-01-01  115171795      8.8277   
5266            Chebi: My Fluffy Friend   2023-01-01   97000000      1.5355   
1118                  Where Am I Going?   2016-01-01   75173000      8.2067   
11314                    Attaly Battaly   2017-01-01   55000000      0.4891   
3584                          Tolo Tolo   2020-01-01   52186426      4.5759   
11578                   Son of a Rich 2   2024-01-01   44459991      2.9492   
17712         The Bremen Town Musicians   2024-01-01   34884591      1.5741   
11303                      Weld Batenha   2017-01-01   30000000      0.8774   
3319                            The Dry   2021-01-01   15576286      3.4699   
465      Three Heroes and Julius Caesar   2015-01-01   13845029      1.1755   
1674                   Mister Happiness   2017-01-01   10748185      9.2749   
1839            

In [13]:
# 1. Create the reduced dataset by excluding the Jan 1st artifacts
#    The '~' operator inverts the boolean mask (selects everything that is NOT Jan 1)
df_reduced = df_raw[~df_raw['is_jan_1']].copy()

# 2. Drop the helper column 'is_jan_1' as it is no longer needed in the clean file
if 'is_jan_1' in df_reduced.columns:
    df_reduced = df_reduced.drop(columns=['is_jan_1'])

# 3. Define the output filename
output_filename = 'movies_no_jan_1.csv'

# 4. Write to disk (index=False prevents writing the row numbers as a separate column)
df_reduced.to_csv(output_filename, index=False)

print(f"Successfully saved reduced dataset to '{output_filename}'.")
print(f"Original row count: {len(df_raw)}")
print(f"New row count:      {len(df_reduced)}")
print(f"Rows dropped:       {len(df_raw) - len(df_reduced)}")


Successfully saved reduced dataset to 'movies_no_jan_1.csv'.
Original row count: 26191
New row count:      23998
Rows dropped:       2193
